# DDoS dataset analysis

L'obiettivo di questo progetto è analizzare un _dataset_ contenente informazioni relative sia ad attacchi DDoS che a normale traffico.
Questo per poter realizzare un'applicazione capace di distinguere il traffico sospetto da quello legittimo e poter quindi tempestivamente bloccare i tentativi di attacco.

## Descrizione del dataset

Il _dataset_ è stato tratto da un _paper_ realizzato dalla "University of New South Wales", in Australia.
Il _paper_ è stato pubblicato su [Science Direct](https://www.sciencedirect.com/science/article/abs/pii/S0167739X18327687) ed è disponibile su [ResearchGate](https://www.researchgate.net/publication/328736466_Towards_the_Development_of_Realistic_Botnet_Dataset_in_the_Internet_of_Things_for_Network_Forensic_Analytics_Bot-IoT_Dataset) in _preprint_.
Come è scritto nello stesso, l'obiettivo del _paper_ era la realizzazione del _dataset_ stesso, rispettando le condizioni di massimo realismo possibile del traffico generato e della configurazione dell'ambiente in cui gli attacchi simulati sono stati svolti.

Il _paper_ indica anche il fatto che sono stati generate diverse tipologie di attacco, ma noi considereremo solamente quelle inerenti agli attacchi di tipo "Distributed Denial of Service", o "DDoS" in breve.
Il _software_ utilizzato per effettuare le catture dei pacchetti è stato [Argus](https://openargus.org/), la cui documentazione, nonché i suoi [esempi d'uso](https://openargus.org/using-argus), indicano come sono costruiti i _record_ che l'applicazione salva nel momento nel quale viene fatta una cattura di rete.

Ogni _record_ è il risultato di un raggruppamento di più pacchetti che svolgono la stessa funzione all'interno di una specifica connessione, o _flow_.
Ad esempio, un _record_ può contenere i pacchetti utilizzati dal protocollo "TCP" per effettuare l'_handshake_ con un'altro nodo di rete, il corpo della trasmissione, oppure la chiusura finale.
Per questo motivo ogni _record_, oltre a contenere informazioni capaci di identificare sorgente e destinazione della connessione, contengono anche dati derivanti dall'aggregazione sulle informazioni di più pacchetti.
Infine, dacché è possibile risalire dai _record_ alle singole connessioni, così come esplicitato nel _paper_ stesso, sono presenti anche già informazioni di aggregazione su alcuni parametri tra più record, informazioni che ci aspettiamo siano replicate uguali tra tutti i _record_ coinvolti.

Il sito in cui il _dataset_ è stato pubblicato è [questo](https://research.unsw.edu.au/projects/bot-iot-dataset), mentre il _download_ diretto dei file può essere fatto dalla [cartella](https://cloudstor.aarnet.edu.au/plus/s/umT99TnxvbpkkoE?path=%2FLabelling) di un servizio _cloud_ della UNSW.
Purtroppo, non è possibile effettuare il _download_ diretto dei file.
I _file_ che sono stati utilizzati in questo progetto sono quelli denominati "DDoS_HTTP.csv", "DDoS_TCP.csv" e "DDoS_UDP.csv".

### Descrizione dei file

I tre _file_ che sono stati utilizzati contengono tre diverse sotto-categorie di attacchi, ovvero attacchi che inviano messaggi "HTTP", segmenti "TCP" e datagrammi "UDP".
Non siamo interessati a tenere conto di questa distinzione, tanto più che tutti e tre i _file_ essendo stati generati dallo stesso _tool_, possiedono lo stesso formato, "CSV", e gli stessi campi.

I campi presenti in ciascun file sono i seguenti:

* "stime": la data e l'ora di ricezione del primo pacchetto del _record_
* "flgs": le _flag_ dello stato della connessione presenti nei pacchetti del _record_
* "proto": il protocollo di livello di rete utilizzato dai pacchetti del _record_
* "saddr": l'indirizzo IP dell'interfaccia sorgente dei pacchetti del _record_
* "sport": la porta dell'interfaccia sorgente dei pacchetti del _record_
* "dir": la direzione del flusso dati, da sorgente a destinazione, viceversa o bidirezionale
* "daddr": l'indirizzo IP dell'interfaccia destinazione dei pacchetti del _record_
* "dport": la porta dell'interfaccia destinazione dei pacchetti del _record_
* "pkts": il numero di pacchetti aggregati dal _record_
* "bytes": la somma dei _byte_ dei pacchetti aggregati
* "state": lo stato della connessione per i pacchetti aggregati dal _record_
* "srcid": l'identificatore usato dal _tool_ "Argus" per identificare la sorgente dati
* "ltime": la data e l'ora di ricezione dell'ultimo pacchetto del _record_
* "seq": il numero di sequenza che il _tool_ "Argus" ha assegnato al _record_
* "dur": la durata totale del _record_ 
* "mean": la durata media dei _record_ aggregati
* "stddev": la deviazione standard dei _record_ aggregati
* "smac": l'indirizzo MAC della sorgente dei pacchetti del _record_
* "dmac": l'indirizzo MAC della destinazione dei pacchetti del _record_
* "sum": la somma delle durate dei _record_ aggregati
* "min": il minimo delle durate dei _record_ aggregati_
* "max": il massimo delle durate dei _record_ aggregati
* "soui": lo "Organizationally Unique Identifier" dell'indirizzo MAC della sorgente dei pacchetti del _record_
* "doui": lo "Organizationally Unique Identifier" dell'indirizzo MAC della destinazione dei pacchetti del _record_
* "sco": il "Country Code" associato all'indirizzo IP della sorgente dei pacchetti nel _record_
* "dco": il "Country Code" associato all'indirizzo IP della destinazione dei pacchetti nel _record_
* "spkts": il numero di pacchetti inviati dalla sorgente alla destinazione in questo _record_
* "dpkts": il numero di pacchetti inviati dalla destinazione alla sorgente in questo _record_
* "sbytes": il numero di _byte_ inviati dalla sorgente alla destinazione in questo _record_
* "dbytes": il numero di _byte_ inviati dalla destinazione alla sorgente in questo _record_
* "rate": i pacchetti al secondo inviati in questo _record_
* "srate": i pacchetti al secondo inviati dalla sorgente alla destinazione in questo _record_
* "drate": i pacchetti al secondo inviati dalla destinazione alla sorgente in questo _record_
* "record": questa feature non è spiegata all'interno del _paper_ né tantomeno nella documentazione di "Argus"
* "attack: se il _record_ è parte di un attacco o meno
* "category": la categoria dell'attacco
* "subcategory": la specifica sotto-categoria dell'attacco

I campi che abbiamo utilizzato nell'analisi sono stati:

* stime
* proto
* saddr
* sport
* dir
* daddr
* dport
* ltime
* dur
* spkts
* dpkts
* sbytes
* dbytes
* srate
* drate
* attack

## Preparazione dei dati

Per effettuare la preparazione dei dati, innanzitutto configuriamo il _kernel_ Spark che utilizzeremo.

In [1]:
%%configure -f
{
    "executorMemory": "8G", 
    "numExecutors": 2, 
    "executorCores": 3, 
    "conf": {
        "spark.dynamicAllocation.enabled": "false"
    }
}

Dopodiché, definiamo i percorsi dei file che utilizzeremo così come sono stati salvati sul servizio "Amazon S3" ed avviamo una nuova applicazione Spark.

In [3]:
//val bucketName = "unibo-bd2122-nfarabegoli/ddos"
//val bucketName = "unibo-bd2122-mcastellucci/project"

val pathTCPDataset = s"s3a://$bucketName/DDoS_TCP.csv"
val pathUDPDataset = s"s3a://$bucketName/DDoS_UDP.csv"
val pathHTTPDataset = s"s3a://$bucketName/DDoS_HTTP.csv"
val pathPortsDataset = s"s3a://$bucketName/ports.csv"

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketName: String = unibo-bd2122-mcastellucci/project
pathTCPDataset: String = s3a://unibo-bd2122-mcastellucci/project/DDoS_TCP.csv
pathUDPDataset: String = s3a://unibo-bd2122-mcastellucci/project/DDoS_UDP.csv
pathHTTPDataset: String = s3a://unibo-bd2122-mcastellucci/project/DDoS_HTTP.csv
pathPortsDataset: String = s3a://unibo-bd2122-mcastellucci/project/ports.csv
res7: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1655987121598_0003/


A questo punto è possibile costruire l'RDD per intero, in modo tale che contenga i dati di tutti e tre i file che ci interessano.

In [4]:
val dataset = sc.textFile(s"$pathTCPDataset,$pathUDPDataset,$pathHTTPDataset")
val ports = sc.textFile(pathPortsDataset)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dataset: org.apache.spark.rdd.RDD[String] = s3a://unibo-bd2122-mcastellucci/project/DDoS_TCP.csv,s3a://unibo-bd2122-mcastellucci/project/DDoS_UDP.csv,s3a://unibo-bd2122-mcastellucci/project/DDoS_HTTP.csv MapPartitionsRDD[1] at textFile at <console>:31
ports: org.apache.spark.rdd.RDD[String] = s3a://unibo-bd2122-mcastellucci/project/ports.csv MapPartitionsRDD[3] at textFile at <console>:27


A questo punto si tratta di fare _parsing_ dei _record_ del _dataset_.
Per effettuarlo correttamente, teniamo conto delle seguenti informazioni sui formati dei valori nelle singole colonne:

* stime, ltime: il valore è un _timestamp_ in secondi dall'epoca UNIX, anche se è espresso in formato decimale per poter avere la precisione dei millisecondi
* proto: il valore può essere uno tra "tcp", "udp", "arp" e "icmp-v6", ognuno dei quali è associato al corrispondente protocollo, sono però di interesse solamente i _record_ associati ai protocolli TCP e UDP
* saddr, dadd: il valore è un indirizzo IP in formato "dotted decimal notation", può perciò essere salvato come String
* sport, dport: il valore è un intero positivo che può arrivare ad un massimo di 65.536, perciò per poter essere rappresentato in linguaggio scala necessita di essere salvato in un Long
* dir:
* pkts, bytes, spkts, dpkts, sbytes, dbytes: il valore è un intero positivo di cui non è noto il massimo, per cui è logico pensare di salvare il valore in un Long
* dur, rate, srate, drate: il valore è un numero decimale, perciò per mantenere la precisione massima utilizziamo un Double
* attack: il valore può essere "1" nel caso il _record_ appartenga ad un attacco DDoS, "0" in caso contrario

Detto questo, sono state implementati i seguenti Astract Data Types:

In [13]:
import java.time.format.DateTimeFormatter
import java.time.{ Instant, LocalDateTime, ZoneId }
import scala.util.{ Try, Success, Failure }

case class Record(
    startTime: LocalDateTime,
    protocol: String,
    sourceAddress: String,
    sourcePort: Long,
    direction: String,
    destinationAddress: String,
    destinationPort: Long,
    packets: Long,
    bytes: Long,
    endTime: LocalDateTime,
    duration: Double,
    rate: Double,
    isDDoS: Boolean,
)

object Record {

  def apply(r: Seq[String]): Option[Record] =
    (for {
      startTime <- Try(
        Instant.ofEpochMilli((r.head.toDouble * 1000).toLong).atZone(ZoneId.systemDefault()).toLocalDateTime,
      )
      protocol <- if (r(2) == "tcp" || r(2) == "udp") Success(r(2)) else Failure(new IllegalStateException())
      sourceAddress = r(3)
      sourcePort <- Try(r(4).toLong)
      direction = r(5)
      destinationAddress = r(6)
      destinationPort <- Try(r(7).toLong)
      packets <- Try(r(8).toLong)
      bytes <- Try(r(9).toLong)
      endTime <- Try(
        Instant.ofEpochMilli((r(12).toDouble * 1000).toLong).atZone(ZoneId.systemDefault()).toLocalDateTime,
      )
      duration <- Try(r(14).toDouble)
      rate <- Try(r(30).toDouble)
      isDDoS = r(34) == "1"
    } yield new Record(
      startTime,
      protocol,
      sourceAddress,
      sourcePort,
      direction,
      destinationAddress,
      destinationPort,
      packets,
      bytes,
      endTime,
      duration,
      rate,
      isDDoS,
    )).toOption
}

case class PortDescription(
  port: Long,
  protocol: String,
  description: String
)

object PortDescription {

  def apply(r: Seq[String]): Option[PortDescription] = 
    (for {
       port <- Try(r.head.toLong)
       protocol <- Try(r(1).toLowerCase)
       description = r(2)
     } yield new PortDescription(port, protocol, description)
    ).toOption
}

case class Quartiles[T: Numeric](min: T, firstQuartile: T, secondQuartile: T, thirdQuartile: T, max: T)

case class Gaussian(mean: Double, stdDev: Double)

object Functions {

  def gaussian(avg: Double, stdDev: Double)(x: Double): Double =
    1 / (stdDev * math.sqrt(2 * math.Pi)) * math.exp(-0.5 * math.pow(x - avg, 2) / math.pow(stdDev, 2))
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.time.format.DateTimeFormatter
import java.time.{Instant, LocalDateTime, ZoneId}
import scala.util.{Try, Success, Failure}
defined class Record
defined object Record
Companions must be defined together; you may wish to use :paste mode for this.
defined class PortDescription
defined object PortDescription
Companions must be defined together; you may wish to use :paste mode for this.
defined class Quartiles
defined class Gaussian
defined object Functions


In [29]:
import org.apache.spark.{HashPartitioner, SparkContext}
import org.apache.spark.rdd.RDD
import scala.reflect.ClassTag
import scala.math.Numeric.Implicits.infixNumericOps

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.{HashPartitioner, SparkContext}
import org.apache.spark.rdd.RDD
import scala.reflect.ClassTag
import scala.math.Numeric.Implicits.infixNumericOps


Alla definizione è seguito il parsing vero e proprio, che ha tenuto conto del fatto che i tre _file_, essendo in formato CSV, hanno le virgolette che circondano ogni valore di ogni colonna e sono separati dai punti e virgola.
Assieme ai _record_ "legittimi", per così dire, saranno presenti anche le intestazioni dei tre _file_.
Questo però non ci preoccupa perché sappiamo che il _parser_ eliminerà correttamente quelle righe dall'RDD che caricheremo, non avendo lo stesso formato delle altre.
Effettuiamo il _caching_ del _dataset_ perché lo riutilizzeremo più volte e così riusciamo a vedere la sua occupazione in formato non serializzato.
La quantità di memoria occupata è di 8.1GB.

In [8]:
val recordDataset = 
    dataset.
        map(_.replace("\"", "")).
        map(_.split(";")).
        map(Record(_)).
        filter(_.isDefined).
        map(_.get).
        cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

recordDataset: org.apache.spark.rdd.RDD[Record] = MapPartitionsRDD[13] at map at <console>:37


In [7]:
val portsDataset = 
    ports.
        map(_.replace("\"", "")).
        map(_.split(",")).
        map(PortDescription(_)).
        filter(_.isDefined).
        map(_.get).
        cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

portsDataset: org.apache.spark.rdd.RDD[PortDescription] = MapPartitionsRDD[8] at map at <console>:37


## Query

### Numero totale di _record_ nel _dataset_

In [12]:
val recordDatasetSize = recordDataset.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

recordDatasetSize: Long = 38532503


Il dataset così caricato e ripulito è formato da 38.532.503 _record_.

### Percentuale dei _record_ associati ad attacchi DDoS

Con questa query si vuole calcolare la percentuale di _record_ che appartengono ad attacchi DDoS nel _dataset_ e così derivare anche il numero di _record_ che __non__ appartengono ad attacchi DDoS, ma a traffico legittimo.

In [ ]:
val ddosCount = 
    recordDataset.
        map(r => if (r.isDDoS) (1, 0) else (0, 1)).
        reduce(_ + _)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ddosCount: (Int, Int) = (38531238,1265)


La query prende in input l'intero _dataset_ e produce due Long: il primo è il numero di _record_ appartenenti ad attacchi DDoS, il secondo è il numero di _record_ appartenenti a traffico legittimo.

La query ha completato in media con un tempo di 52 secondi prendendo in input 12.7 GB di dati.

L'implementazione scelta non fa altro che trasformare ogni _record_ in una coppia di valori, dove il primo rappresenta il conteggio dei _record_ associati ad attacchi, il secondo a traffico legittimo, che avrà il valore 1 nella posizione corrispondente a quale tipo di traffico il _record_ stesso appartiene, il valore 0 nell'altra.
Dopodiché, viene compiuta un'operazione di _reduce_ che somma i valori nelle corrispondenti posizioni.

È stata tentata una variante ottimizzata dove si raggruppavano i _record_ per tipologia di traffico di appartenenza, ma aveva un tempo di esecuzione comparabile a questa implementazione, pur raddoppiando il numero di _task_, dopodiché sulla nuova implementazione si è tentato di forzare il partizionamento a 2 partizioni tramite `HashPartioner`, ma ha peggiorato il tempo di esecuzione.
Evidentemente lo _shuffling_ derivato dal ripartizionamento non è giustificato da un eventuale riduzione del tempo di esecuzione della _reduce_.

In [12]:
val ddosPercentage = ddosCount._1 / recordDatasetSize.toDouble * 100
val legitPercentage = ddosCount._2 / recordDatasetSize.toDouble * 100

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ddosPercentage: Double = 99.99671705728538
legitPercentage: Double = 0.0032829427146219906


Quello che è possibile dedurre da questa _query_ è che il _dataset_ contiene quasi esclusivamente _record_ inerenti ad attacchi DDoS, il 99,997%. 
Per qusto motivo, occorrerà tenere in conto di questa differenza di peso tra i due tipi di traffico.

<div align="center">
    <img src="images/total_pie.png" width="500"/>
</div>

### Percentuale dei protocolli coinvolti negli attacchi DDoS

Con questa query si vuole calcolare la percentuale con cui ciascun protocollo, ovvero "TCP" e "UDP", compare nei _record_ che appartengono ad attacchi DDoS nel _dataset_.

In [13]:
val ddosByProtocol = 
    recordDataset.
        filter(_.isDDoS).
        map(_.protocol).
        countByValue()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ddosByProtocol: scala.collection.Map[String,Long] = Map(tcp -> 19566842, udp -> 18964396)


# INSERIRE SPIEGAZIONE QUI

In [14]:
val tcpPercentage = ddosByProtocol("tcp") / recordDatasetSize.toDouble * 100
val udpPercentage = ddosByProtocol("udp") / recordDatasetSize.toDouble * 100

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

tcpPercentage: Double = 50.780095962102436
udpPercentage: Double = 49.216621095182944


Come si può notare, il peso che entrambi i protocolli hanno negli attacchi DDoS è all'incirca lo stesso.
Non è perciò possibile aspettarsi che un certo segmento sia potenzialmente più o meno pericoloso a seconda del tipo di protocollo di livello di trasporto utilizzato.

<div align="center">
    <img src="images/protocol-chart.png" width="500"/>
</div>

### Percentuale dei servizi presi di mira dagli attacchi DDoS

Con questa query si vuole calcolare la percentuale con cui ciascun servizio è stato preso di mira dagli attacchi DDoS.
Per servizio intendiamo la combinazione di porta e protocollo di livello di rete, che di norma sono associati a servizi predefiniti.
Ad esempio, il protocollo TCP e la porta 80 sono di norma associati ad un server HTTP, anche se nulla toglie che è possibile che sia presente un altro servizio in ascolto.
Certamente, un attaccante può studiare prima il sistema per capire quali servizi sono attivi, ma in mancanza di altre informazioni disponibili ricade su ciò che è vero per default.

In [ ]:
val portsCountByJoin = 
    recordDataset.
        filter(_.isDDoS).
        map(r => ((r.destinationPort, r.protocol), 1)).
        reduceByKey(_ + _).
        join(portsDataset.map(d => ((d.port, d.protocol), d.description))).
        sortBy(_._2, ascending = false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

portsCountByJoin: org.apache.spark.rdd.RDD[((Long, String), (Int, String))] = MapPartitionsRDD[25] at sortBy at <console>:41


# INSERIRE SPIEGAZIONE QUI

In [64]:
val totalPortsCount = portsCountByJoin.map(_._2._1).reduce(_ + _)

println("| Service  | Description | Total records | Percentage |")
println("|----------|-------------|---------------|------------|")
(portsCountByJoin.
    take(2).
    map { case ((port, protocol), (count, desc)) => 
        (s"$port - $protocol", f"$desc%-11s", f"$count%-13d", f"${(count.toDouble / totalPortsCount * 100).toString.substring(0, 5) + "%" }%-10s")
    }.
    toSeq :+ 
    (
        "Other   ", 
        "N/A        ", 
        f"${totalPortsCount - portsCountByJoin.take(2).map(_._2._1).sum}%-13d", 
        f"${((totalPortsCount - portsCountByJoin.take(2).map(_._2._1).sum).toDouble / totalPortsCount * 100).toString.substring(0, 4) + "%"}%-10s")
    ).
    foreach(t => println(s"| ${t._1} | ${t._2} | ${t._3} | ${t._4} |"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

totalPortsCount: Int = 38504055
| Service  | Description | Total records | Percentage |
|----------|-------------|---------------|------------|
| 80 - tcp | HTTP        | 19529331      | 50.72%     |
| 80 - udp | HTTP        | 18964396      | 49.25%     |
| Other    | N/A         | 10328         | 0.02%      |


# INSERIRE COMMENTO QUI

<div align="center">
    <img src="images/ports-chart.png" width="500"/>
</div>

### Quantità di traffico DDoS in byte rispetto al totale

# DESCRIZIONE QUI

In [65]:
val ddosTrafficByIP =
    recordDataset.
    filter(_.isDDoS).
    map(r => (r.destinationAddress, r.bytes)).
    reduceByKey(_ + _).
    map { case (ip, traffic) => (ip, traffic / 1024.toDouble) }.
    sortBy(_._2, ascending = false).
    take(5).
    toMap

val trafficByIP =
    recordDataset.
    map(r => (r.destinationAddress, r.bytes)).
    reduceByKey(_ + _).
    map { case (ip, traffic) => (ip, traffic / 1024.toDouble) }.
    collect().
    toMap

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ddosTrafficByIP: scala.collection.immutable.Map[String,Double] = Map(192.168.100.148 -> 7457.1875, 192.168.100.5 -> 5259153.908203125, 192.168.100.7 -> 3382470.841796875, 192.168.100.3 -> 8999801.200195312, 192.168.100.6 -> 2398495.455078125)
trafficByIP: scala.collection.immutable.Map[String,Double] = Map(184.85.248.65 -> 0.3251953125, 34.213.81.108 -> 12.1943359375, 192.168.100.148 -> 7457.1875, 192.5.5.241 -> 1.259765625, 192.168.100.1 -> 1.9775390625, 96.7.49.66 -> 0.3251953125, 224.0.0.251 -> 0.46875, 192.168.100.255 -> 10.640625, 172.217.25.170 -> 13.1513671875, 192.168.100.147 -> 4621.00390625, 192.36.148.17 -> 4.90234375, 116.206.80.123 -> 0.87890625, ff02::fb -> 0.52734375, 116.206.83.243 -> 1.0546875, 192.168.100.46 -> 275956.6396484375, 192.33.14.30 -> 7.7900390625, 192.168.100.5 -> 7079893.10546875, 192.168.100.55 -> 329540.4853515625, 216.239.36.10 -> 0.5888671875, 193.108.91.240 -> 0.3251953125, 205.251.196.236 -> 1.2275390625, 27.124.125.251 -> 0.52734375, 202.12.27.33 -

# INSERIRE SPIEGAZIONE QUI

In [66]:
val totalImportantTraffic = trafficByIP.filterKeys(ddosTrafficByIP.keySet(_))
val totalImportantTrafficSorted = totalImportantTraffic.toSeq.sortBy(_._1)
val ddosTrafficSorted = ddosTrafficByIP.toSeq.sortBy(_._1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

totalTrafficOnlyDDos: scala.collection.immutable.Map[String,Double] = Map(192.168.100.148 -> 7457.1875, 192.168.100.5 -> 7079893.10546875, 192.168.100.7 -> 3382470.841796875, 192.168.100.3 -> 1.0039371459960938E7, 192.168.100.6 -> 2398495.455078125)
a1: Seq[(String, Double)] = Vector((192.168.100.148,7457.1875), (192.168.100.3,1.0039371459960938E7), (192.168.100.5,7079893.10546875), (192.168.100.6,2398495.455078125), (192.168.100.7,3382470.841796875))
a2: Seq[(String, Double)] = Vector((192.168.100.148,7457.1875), (192.168.100.3,8999801.200195312), (192.168.100.5,5259153.908203125), (192.168.100.6,2398495.455078125), (192.168.100.7,3382470.841796875))


# INSERIRE COMMENTO QUI

<div align="center">
    <img src="images/ddos-traffic.png" width="500"/>
</div>

### Conteggio dei flussi catturati nel dataset

Un flusso lo si identifica da (sourceIp, sourcePort, destinationIp, destinationPort)

In [11]:
val flowsDataset =
    recordDataset.
    filter(_.duration > 0.0).
    map(r =>
      (
        (r.sourceAddress, r.sourcePort, r.destinationAddress, r.destinationPort, r.protocol),
        (r.isDDoS, r.duration, r.packets, r.bytes),
      ),
    ).
    reduceByKey { case ((isDDoS1, duration1, packets1, bytes1), (isDDoS2, duration2, packets2, bytes2)) =>
      (isDDoS1 || isDDoS2, duration1 + duration2, packets1 + packets2, bytes1 + bytes2)
    }.
    map { case (_, (isDDoS, duration, packets, bytes)) => (isDDoS, packets / duration, bytes / duration) }.
    cache()

val (ddosPacketsRateSum, ddosBytesRateSum, ddosCount, legitPacketsRateSum, legitBytesRateSum, legitCount) =
    flowsDataset.
    map { case (isDDoS, packetsRate, bytesRate) =>
      if (isDDoS) (packetsRate, bytesRate, 1, 0.0, 0.0, 0) else (0.0, 0.0, 0, packetsRate, bytesRate, 1)
    }.
    reduce { 
      case(
        (ddosPacketsRate1, ddosBytesRate1, ddosCount1, legitPacketsRate1, legitBytesRate1, legitCount1), 
        (ddosPacketsRate2, ddosBytesRate2, ddosCount2, legitPacketsRate2, legitBytesRate2, legitCount2)
      ) => (
        ddosPacketsRate1 + ddosPacketsRate2, 
        ddosBytesRate1 + ddosBytesRate2, 
        ddosCount1 + ddosCount2, 
        legitPacketsRate1 + legitPacketsRate2, 
        legitBytesRate1 + legitBytesRate2,
        legitCount1 + legitCount2
      )  
    }

val ddosPacketsRateMean = ddosPacketsRateSum / ddosCount
val ddosBytesRateMean = ddosBytesRateSum / ddosCount
val legitPacketsRateMean = legitPacketsRateSum / legitCount
val legitBytesRateMean = legitBytesRateSum / legitCount

val ddosPacketsRateMeanBroadcast = sc.broadcast(ddosPacketsRateMean)
val ddosBytesRateMeanBroadcast = sc.broadcast(ddosBytesRateMean)
val legitPacketsRateMeanBroadcast = sc.broadcast(legitPacketsRateMean)
val legitBytesRateMeanBroadcast = sc.broadcast(legitBytesRateMean)

val (ddosPacketsRateDiff, ddosBytesRateDiff, legitPacketsRateDiff, legitBytesRateDiff) =
    flowsDataset.
    map { case (isDDoS, packetsRate, bytesRate) =>
      if (isDDoS)
        (
         math.pow(packetsRate - ddosPacketsRateMeanBroadcast.value, 2),
         math.pow(bytesRate - ddosBytesRateMeanBroadcast.value, 2),
         0.0,
         0.0,
        )
      else
        (
         0.0,
         0.0,
         math.pow(packetsRate - legitPacketsRateMeanBroadcast.value, 2),
         math.pow(bytesRate - legitBytesRateMeanBroadcast.value, 2),
        )
    }.
    reduce {
      case(
        (ddosPacketsRateDiff1, ddosBytesRateDiff1, legitPacketsRateDiff1, legitBytesRateDiff1),
        (ddosPacketsRateDiff2, ddosBytesRateDiff2, legitPacketsRateDiff2, legitBytesRateDiff2)
      ) => (
        ddosPacketsRateDiff1 + ddosPacketsRateDiff2,
        ddosBytesRateDiff1 + ddosBytesRateDiff2,
        legitPacketsRateDiff1 + legitPacketsRateDiff2,
        legitBytesRateDiff1 + legitBytesRateDiff2
      )
    }

val ddosPacketsRateStdDev = math.sqrt(ddosPacketsRateDiff / ddosCount)
val ddosBytesRateStdDev = math.sqrt(ddosBytesRateDiff / ddosCount)
val legitPacketsRateStdDev = math.sqrt(legitPacketsRateDiff / legitCount)
val legitBytesRateStdDev = math.sqrt(legitBytesRateDiff / legitCount)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

flowsDataset: org.apache.spark.rdd.RDD[(Boolean, Double, Double)] = MapPartitionsRDD[23] at map at <console>:41
ddosPacketsRateSum: Double = 584305.5830716968
ddosBytesRateSum: Double = 5.646841374913517E7
ddosCount: Int = 2132236
legitPacketsRateSum: Double = 31059.85973989538
legitBytesRateSum: Double = 5124031.632252166
legitCount: Int = 89
ddosPacketsRateMean: Double = 0.2740341984056628
ddosBytesRateMean: Double = 26.483191236399332
legitPacketsRateMean: Double = 348.9871880887121
legitBytesRateMean: Double = 57573.38912642883
ddosPacketsRateMeanBroadcast: org.apache.spark.broadcast.Broadcast[Double] = Broadcast(11)
ddosBytesRateMeanBroadcast: org.apache.spark.broadcast.Broadcast[Double] = Broadcast(12)
legitPacketsRateMeanBroadcast: org.apache.spark.broadcast.Broadcast[Double] = Broadcast(13)
legitBytesRateMeanBroadcast: org.apache.spark.broadcast.Broadcast[Double] = Broadcast(14)
ddosPacketsRateDiff: Double = 51875.59647335778
ddosBytesRateDiff: Double = 4.550126845388437E8
legi

# INSERIRE SPIEGAZIONE QUI

# INSERIRE COMMENTO QUI

<div align="center">
    <div>
        <img src="images/ddos-flow-packets-rate.png" width="40%"/>
        <img src="images/legit-flow-packets-rate.png" width="40%"/>
    </div>
    <div>
        <img src="images/ddos-flow-bytes-rate.png" width="40%"/>
        <img src="images/legit-flow-bytes-rate.png" width="500"/>
    </div>
</div>

## Statistiche temporali su attacchi 

# INSERIRE DESCRIZIONE QUI

In [19]:
def minDate(date1: LocalDateTime, date2: LocalDateTime): LocalDateTime = {
    if (date1.isBefore(date2)) date1 else date2
}

def maxDate(date1: LocalDateTime, date2: LocalDateTime): LocalDateTime = {
    if (date1.isAfter(date2)) date1 else date2
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

minDate: (date1: java.time.LocalDateTime, date2: java.time.LocalDateTime)java.time.LocalDateTime
maxDate: (date1: java.time.LocalDateTime, date2: java.time.LocalDateTime)java.time.LocalDateTime


# INSERIRE SPIEGAZIONE QUI

Dalla query seguente andiamo a determinare la data di inizio dell'attacco e la data di fine

In [22]:
val minMaxDate = recordDataset.filter(_.isDDoS).map(r => (r.startTime, r.startTime)).reduce {
    case ((accMax, x), (r1, r2)) => (maxDate(accMax, r1), minDate(accMin, r2))
}

val startTime = minMaxDate._2
val endTime = minMaxDate._1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

minMaxDate: (java.time.LocalDateTime, java.time.LocalDateTime) = (2018-06-04T09:22:59.512,2018-06-04T07:02:01.762)
startTime: java.time.LocalDateTime = 2018-06-04T07:02:01.762
endTime: java.time.LocalDateTime = 2018-06-04T09:22:59.512


# INSERIRE COMMENTO QUI

### Statistiche d'ordine e distribuzione delle variabili "packets", "bytes", "rate" e "byteRate"

# INSERIRE DESCRIZIONE QUI

In [30]:
def getStatistics[T: Numeric: Ordering: ClassTag](dataset: RDD[T]): Quartiles[T] = {
    val size = dataset.count()
    val quartiles = 
        dataset.
            sortBy(v => v).
            zipWithIndex().
            filter { case (_, index) =>
                index == 0 || index == size / 4 || index == size / 2 || index == size * 3 / 4 || index == size - 1
            }.
            collect()
    quartiles.sortBy(_._2).map(_._1) match {
      case Array(min, first, second, third, max) => Quartiles(min, first, second, third, max)
    }
  }

def getGaussian[T: Numeric](sc: SparkContext, rdd: RDD[T]): Gaussian = {
    val (sum, count) = rdd.map(r => (r, 1)).reduce{ case((sum1, count1), (sum2, count2)) => (sum1 + sum2, count1 + count2) }
    val mean = sum.toDouble() / count
    val meanBroadcast = sc.broadcast(mean)

    val stdDev = math.sqrt(rdd.map(r => math.pow(r.toDouble() - meanBroadcast.value, 2)).sum() / count)
    Gaussian(mean, stdDev)
}

def cleanByIQR[T: Numeric: Ordering](sc: SparkContext, rdd: RDD[T], quartiles: Quartiles[T]): RDD[T] = {
    val rangeBroadcast = sc.broadcast(
        (
            quartiles.firstQuartile.toDouble() - 1.5 * (quartiles.thirdQuartile - quartiles.firstQuartile).toDouble(),
            quartiles.thirdQuartile.toDouble() + 1.5 * (quartiles.thirdQuartile - quartiles.firstQuartile).toDouble(),
        ),
    )
    rdd.filter(r => r.toDouble() > rangeBroadcast.value._1 && r.toDouble() < rangeBroadcast.value._2)
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

getStatistics: [T](dataset: org.apache.spark.rdd.RDD[T])(implicit evidence$1: Numeric[T], implicit evidence$2: Ordering[T], implicit evidence$3: scala.reflect.ClassTag[T])Quartiles[T]
getGaussian: [T](sc: org.apache.spark.SparkContext, rdd: org.apache.spark.rdd.RDD[T])(implicit evidence$1: Numeric[T])Gaussian
cleanByIQR: [T](sc: org.apache.spark.SparkContext, rdd: org.apache.spark.rdd.RDD[T], quartiles: Quartiles[T])(implicit evidence$1: Numeric[T], implicit evidence$2: Ordering[T])org.apache.spark.rdd.RDD[T]


# INSERIRE SPIEGAZIONE QUI

In [ ]:
val ddosDataset = recordDataset.filter(_.isDDoS).cache()
val legitDataset = recordDataset.filter(!_.isDDoS).cache()

val packetsDDoS = ddosDataset.map(_.packets)
val packetsLegit = legitDataset.map(_.packets)
val bytesDDoS = ddosDataset.map(_.bytes)
val bytesLegit = legitDataset.map(_.bytes)
val rateDDoS = ddosDataset.map(_.rate)
val rateLegit = legitDataset.map(_.rate)
val bytesRateDDoS = ddosDataset.map(r => r.bytes / r.duration)
val bytesRateLegit = legitDataset.map(r => r.bytes / r.duration)

val packetQuartileDDoS = getStatistics(packetsDDoS)
val packetQuartileLegit = getStatistics(packetsLegit)
val bytesQuartileDDoS = getStatistics(bytesDDoS)
val bytesQuartileLegit = getStatistics(bytesLegit)
val rateQuartileDDoS = getStatistics(rateDDoS)
val rateQuartileLegit = getStatistics(rateLegit)
val bytesRateQuartileDDoS = getStatistics(bytesRateDDoS)
val bytesRateQuartileLegit = getStatistics(bytesRateLegit)

val packetDDoSGaussian = getGaussian(sc, cleanByIQR(sc, packetsDDoS, packetQuartileDDoS))
val packetLegitGaussian = getGaussian(sc, cleanByIQR(sc, packetsLegit, packetQuartileLegit))
val bytesQuartileDDoSGaussian = getGaussian(sc, cleanByIQR(sc, bytesDDoS, bytesQuartileDDoS))
val bytesQuartileLegitGaussian = getGaussian(sc, cleanByIQR(sc, bytesLegit, bytesQuartileLegit))
val rateDDoSGaussian = getGaussian(sc, cleanByIQR(sc, rateDDoS, rateQuartileDDoS))
val rateLegitGaussian = getGaussian(sc, cleanByIQR(sc, rateLegit, rateQuartileLegit))
val bytesRateDDoSGaussian = getGaussian(sc, cleanByIQR(sc, bytesRateDDoS, bytesRateQuartileDDoS))
val bytesRateLegitGaussian = getGaussian(sc, cleanByIQR(sc, bytesRateLegit, bytesRateQuartileLegit))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# INSERIRE COMMENTO QUI

<div align="center">
    <div>
        <img src="images/packets-ddos.png" width="40%"/>
        <img src="images/packets-legit.png" width="40%"/>
    </div>
    <div>
        <img src="images/bytes-ddos.png" width="40%"/>
        <img src="images/bytes-legit.png" width="40%"/>
    </div>
    <div>
        <img src="images/rates-ddos.png" width="40%"/>
        <img src="images/rates-legit.png" width="40%"/>
    </div>
    <div>
        <img src="images/bytesRate-ddos.png" width="40%"/>
        <img src="images/bytesRate-legit.png" width="40%"/>
    </div>
</div>

## Costruzione della metrica